In [18]:
import torch
import torchvision
import matplotlib.pyplot as plt
import imp
from perceiver_basic import model
from torch.utils.data import DataLoader

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
class FMPreprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.shape = (1, 28, 28)
        self.encoding = self._generate_encoding()
        self.encoding = torch.nn.Parameter(self.encoding)
        self.encoding.requires_grad = False

    def _generate_encoding(self):
        dims = len(self.shape)-1 #minus batch dim
        encoding = torch.zeros([*self.shape, dims])
        for i in range(encoding[0].shape[0]):
            for j in range(encoding[0].shape[1]):
                encoding[0,i,j][0] = i * 2.0 / encoding[0].shape[0] - 1.0
                encoding[0,i,j][1] = i * 2.0 / encoding[0].shape[0] - 1.0
        return encoding

    def forward(self, in_val):
        x = in_val.squeeze().clone()
        x = x.unsqueeze(-1)
        if in_val.shape[0] == 1:
            x = x.unsqueeze(0)
        batch_count = in_val.shape[0]
        dims = [-1 for _ in range(len(self.encoding.shape))]
        dims[0] = batch_count
        encoding = self.encoding.expand(*dims)
        x = torch.cat([x, encoding], dim=-1) #concatenating positional encodings
        #x = torch.mul(x, encoding) #scaled positional encooding

        #maintain batch and pixel codes, pixels will be in sequential form
        x = torch.flatten(x, start_dim=1, end_dim=-2)

        return x

In [21]:
pre = FMPreprocess()
garbo = torch.randn([5, 28, 28])
pre(garbo).shape

torch.Size([5, 784, 3])

In [22]:
imp.reload(model)
class PerceiverClassify(torch.nn.Module):
    def __init__(self, latent_dim, heads, wide_factor, latent_count, repeat_count=5, p_dropout=0.1):
        super().__init__()
        self.preprocess = FMPreprocess()
        out_dim = (1,32)
        in_channels = len(self.preprocess.shape) #subtract one for batch dim, but add one for the pixel dim
        self.perceiver = model.PerceiverInternal(in_channels, latent_dim, out_dim, heads, wide_factor, latent_count, repeat_count, p_dropout)
        
        self.lin_out = torch.nn.Linear(32, 10)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        #x = x / self.range
        x = self.preprocess(x)

        x = self.perceiver(x)
      
        # x.shape <- (batch_count, 1, 32)
        x = x.squeeze()
        x = self.lin_out(x)
        x = self.sigmoid(x) * 2.0 - 1.0

        return x

In [23]:
per = PerceiverClassify(latent_dim=(8, 8), heads=8, wide_factor=4, latent_count=6)
garbo = torch.randn([58, 28, 28])
per(garbo).shape

torch.Size([58, 10])

In [24]:
BATCH_SIZE = 16
EPOCHS = 10

In [25]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.FashionMNIST('./fm_set', transform=transforms, download=True)
testset = torchvision.datasets.FashionMNIST('./fm_set', train=False, transform=transforms, download=True)

In [26]:
torch.min(dataset[0][0])

tensor(0.)

In [27]:
dataloader = DataLoader(dataset, num_workers=2, batch_size=BATCH_SIZE)

In [28]:
perceiver = PerceiverClassify(latent_dim=(16, 16), heads=8, wide_factor=4, latent_count=8)
perceiver.to(device)
optimizer = torch.optim.Adam(perceiver.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [29]:
garbo = torch.randn([1, 28, 28])
garbo = garbo.to(device)
perceiver(garbo).shape

torch.Size([10])

In [30]:
for epoch in range(EPOCHS):
    print(f"Entering epoch {epoch}")
    for batch_no, (features, labels) in enumerate(dataloader):
        features = features.to(device)
        labels = labels.to(device)

        predictions = perceiver(features)
        
        loss = criterion(predictions, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Iteration - {batch_no}')

Entering epoch 0
Iteration - 0
Iteration - 1
Iteration - 2
Iteration - 3
Iteration - 4
Iteration - 5
Iteration - 6
Iteration - 7
Iteration - 8
Iteration - 9
Iteration - 10
Iteration - 11
Iteration - 12
Iteration - 13
Iteration - 14
Iteration - 15
Iteration - 16
Iteration - 17
Iteration - 18
Iteration - 19
Iteration - 20
Iteration - 21
Iteration - 22
Iteration - 23
Iteration - 24
Iteration - 25
Iteration - 26
Iteration - 27
Iteration - 28
Iteration - 29
Iteration - 30
Iteration - 31
Iteration - 32
Iteration - 33
Iteration - 34
Iteration - 35
Iteration - 36
Iteration - 37
Iteration - 38
Iteration - 39
Iteration - 40
Iteration - 41
Iteration - 42
Iteration - 43
Iteration - 44
Iteration - 45
Iteration - 46
Iteration - 47
Iteration - 48
Iteration - 49
Iteration - 50
Iteration - 51
Iteration - 52
Iteration - 53
Iteration - 54
Iteration - 55
Iteration - 56
Iteration - 57
Iteration - 58
Iteration - 59
Iteration - 60
Iteration - 61
Iteration - 62
Iteration - 63
Iteration - 64
Iteration - 65
Ite

Iteration - 520
Iteration - 521
Iteration - 522
Iteration - 523
Iteration - 524
Iteration - 525
Iteration - 526
Iteration - 527
Iteration - 528
Iteration - 529
Iteration - 530
Iteration - 531
Iteration - 532
Iteration - 533
Iteration - 534
Iteration - 535
Iteration - 536
Iteration - 537
Iteration - 538
Iteration - 539
Iteration - 540
Iteration - 541
Iteration - 542
Iteration - 543
Iteration - 544
Iteration - 545
Iteration - 546
Iteration - 547
Iteration - 548
Iteration - 549
Iteration - 550
Iteration - 551
Iteration - 552
Iteration - 553
Iteration - 554
Iteration - 555
Iteration - 556
Iteration - 557
Iteration - 558
Iteration - 559
Iteration - 560
Iteration - 561
Iteration - 562
Iteration - 563
Iteration - 564
Iteration - 565
Iteration - 566
Iteration - 567
Iteration - 568
Iteration - 569
Iteration - 570
Iteration - 571
Iteration - 572
Iteration - 573
Iteration - 574
Iteration - 575
Iteration - 576
Iteration - 577
Iteration - 578
Iteration - 579
Iteration - 580
Iteration - 581
Iteratio

Iteration - 1034
Iteration - 1035
Iteration - 1036
Iteration - 1037
Iteration - 1038
Iteration - 1039
Iteration - 1040
Iteration - 1041
Iteration - 1042
Iteration - 1043
Iteration - 1044
Iteration - 1045
Iteration - 1046
Iteration - 1047
Iteration - 1048
Iteration - 1049
Iteration - 1050
Iteration - 1051
Iteration - 1052
Iteration - 1053
Iteration - 1054
Iteration - 1055
Iteration - 1056
Iteration - 1057
Iteration - 1058
Iteration - 1059
Iteration - 1060
Iteration - 1061
Iteration - 1062
Iteration - 1063
Iteration - 1064
Iteration - 1065
Iteration - 1066
Iteration - 1067
Iteration - 1068
Iteration - 1069
Iteration - 1070
Iteration - 1071
Iteration - 1072
Iteration - 1073
Iteration - 1074
Iteration - 1075
Iteration - 1076
Iteration - 1077
Iteration - 1078
Iteration - 1079
Iteration - 1080
Iteration - 1081
Iteration - 1082
Iteration - 1083
Iteration - 1084
Iteration - 1085
Iteration - 1086
Iteration - 1087
Iteration - 1088
Iteration - 1089
Iteration - 1090
Iteration - 1091
Iteration - 10

Iteration - 1519
Iteration - 1520
Iteration - 1521
Iteration - 1522
Iteration - 1523
Iteration - 1524
Iteration - 1525
Iteration - 1526
Iteration - 1527
Iteration - 1528
Iteration - 1529
Iteration - 1530
Iteration - 1531
Iteration - 1532
Iteration - 1533
Iteration - 1534
Iteration - 1535
Iteration - 1536
Iteration - 1537
Iteration - 1538
Iteration - 1539
Iteration - 1540
Iteration - 1541
Iteration - 1542
Iteration - 1543
Iteration - 1544
Iteration - 1545
Iteration - 1546
Iteration - 1547
Iteration - 1548
Iteration - 1549
Iteration - 1550
Iteration - 1551
Iteration - 1552
Iteration - 1553
Iteration - 1554
Iteration - 1555
Iteration - 1556
Iteration - 1557
Iteration - 1558
Iteration - 1559
Iteration - 1560
Iteration - 1561
Iteration - 1562
Iteration - 1563
Iteration - 1564
Iteration - 1565
Iteration - 1566
Iteration - 1567
Iteration - 1568
Iteration - 1569
Iteration - 1570
Iteration - 1571
Iteration - 1572
Iteration - 1573
Iteration - 1574
Iteration - 1575
Iteration - 1576
Iteration - 15

Iteration - 2001
Iteration - 2002
Iteration - 2003
Iteration - 2004
Iteration - 2005
Iteration - 2006
Iteration - 2007
Iteration - 2008
Iteration - 2009
Iteration - 2010
Iteration - 2011
Iteration - 2012
Iteration - 2013
Iteration - 2014
Iteration - 2015
Iteration - 2016
Iteration - 2017
Iteration - 2018
Iteration - 2019
Iteration - 2020
Iteration - 2021
Iteration - 2022
Iteration - 2023
Iteration - 2024
Iteration - 2025
Iteration - 2026
Iteration - 2027
Iteration - 2028
Iteration - 2029
Iteration - 2030
Iteration - 2031
Iteration - 2032
Iteration - 2033
Iteration - 2034
Iteration - 2035
Iteration - 2036
Iteration - 2037
Iteration - 2038
Iteration - 2039
Iteration - 2040
Iteration - 2041
Iteration - 2042
Iteration - 2043
Iteration - 2044
Iteration - 2045
Iteration - 2046
Iteration - 2047
Iteration - 2048
Iteration - 2049
Iteration - 2050
Iteration - 2051
Iteration - 2052
Iteration - 2053
Iteration - 2054
Iteration - 2055
Iteration - 2056
Iteration - 2057
Iteration - 2058
Iteration - 20

...

Iteration - 290
Iteration - 291
Iteration - 292
Iteration - 293
Iteration - 294
Iteration - 295
Iteration - 296
Iteration - 297
Iteration - 298
Iteration - 299
Iteration - 300
Iteration - 301
Iteration - 302
Iteration - 303
Iteration - 304
Iteration - 305
Iteration - 306
Iteration - 307
Iteration - 308
Iteration - 309
Iteration - 310
Iteration - 311
Iteration - 312
Iteration - 313
Iteration - 314
Iteration - 315
Iteration - 316
Iteration - 317
Iteration - 318
Iteration - 319
Iteration - 320
Iteration - 321
Iteration - 322
Iteration - 323
Iteration - 324
Iteration - 325
Iteration - 326
Iteration - 327
Iteration - 328
Iteration - 329
Iteration - 330
Iteration - 331
Iteration - 332
Iteration - 333
Iteration - 334
Iteration - 335
Iteration - 336
Iteration - 337
Iteration - 338
Iteration - 339
Iteration - 340
Iteration - 341
Iteration - 342
Iteration - 343
Iteration - 344
Iteration - 345
Iteration - 346
Iteration - 347
Iteration - 348
Iteration - 349
Iteration - 350
Iteration - 351
Iteratio

Iteration - 805
Iteration - 806
Iteration - 807
Iteration - 808
Iteration - 809
Iteration - 810
Iteration - 811
Iteration - 812
Iteration - 813
Iteration - 814
Iteration - 815
Iteration - 816
Iteration - 817
Iteration - 818
Iteration - 819
Iteration - 820
Iteration - 821
Iteration - 822
Iteration - 823
Iteration - 824
Iteration - 825
Iteration - 826
Iteration - 827
Iteration - 828
Iteration - 829
Iteration - 830
Iteration - 831
Iteration - 832
Iteration - 833
Iteration - 834
Iteration - 835
Iteration - 836
Iteration - 837
Iteration - 838
Iteration - 839
Iteration - 840
Iteration - 841
Iteration - 842
Iteration - 843
Iteration - 844
Iteration - 845
Iteration - 846
Iteration - 847
Iteration - 848
Iteration - 849
Iteration - 850
Iteration - 851
Iteration - 852
Iteration - 853
Iteration - 854
Iteration - 855
Iteration - 856
Iteration - 857
Iteration - 858
Iteration - 859
Iteration - 860
Iteration - 861
Iteration - 862
Iteration - 863
Iteration - 864
Iteration - 865
Iteration - 866
Iteratio

Iteration - 1299
Iteration - 1300
Iteration - 1301
Iteration - 1302
Iteration - 1303
Iteration - 1304
Iteration - 1305
Iteration - 1306
Iteration - 1307
Iteration - 1308
Iteration - 1309
Iteration - 1310
Iteration - 1311
Iteration - 1312
Iteration - 1313
Iteration - 1314
Iteration - 1315
Iteration - 1316
Iteration - 1317
Iteration - 1318
Iteration - 1319
Iteration - 1320
Iteration - 1321
Iteration - 1322
Iteration - 1323
Iteration - 1324
Iteration - 1325
Iteration - 1326
Iteration - 1327
Iteration - 1328
Iteration - 1329
Iteration - 1330
Iteration - 1331
Iteration - 1332
Iteration - 1333
Iteration - 1334
Iteration - 1335
Iteration - 1336
Iteration - 1337
Iteration - 1338
Iteration - 1339
Iteration - 1340
Iteration - 1341
Iteration - 1342
Iteration - 1343
Iteration - 1344
Iteration - 1345
Iteration - 1346
Iteration - 1347
Iteration - 1348
Iteration - 1349
Iteration - 1350
Iteration - 1351
Iteration - 1352
Iteration - 1353
Iteration - 1354
Iteration - 1355
Iteration - 1356
Iteration - 13

Iteration - 1784
Iteration - 1785
Iteration - 1786
Iteration - 1787
Iteration - 1788
Iteration - 1789
Iteration - 1790
Iteration - 1791
Iteration - 1792
Iteration - 1793
Iteration - 1794
Iteration - 1795
Iteration - 1796
Iteration - 1797
Iteration - 1798
Iteration - 1799
Iteration - 1800
Iteration - 1801
Iteration - 1802
Iteration - 1803
Iteration - 1804
Iteration - 1805
Iteration - 1806
Iteration - 1807
Iteration - 1808
Iteration - 1809
Iteration - 1810
Iteration - 1811
Iteration - 1812
Iteration - 1813
Iteration - 1814
Iteration - 1815
Iteration - 1816
Iteration - 1817
Iteration - 1818
Iteration - 1819
Iteration - 1820
Iteration - 1821
Iteration - 1822
Iteration - 1823
Iteration - 1824
Iteration - 1825
Iteration - 1826
Iteration - 1827
Iteration - 1828
Iteration - 1829
Iteration - 1830
Iteration - 1831
Iteration - 1832
Iteration - 1833
Iteration - 1834
Iteration - 1835
Iteration - 1836
Iteration - 1837
Iteration - 1838
Iteration - 1839
Iteration - 1840
Iteration - 1841
Iteration - 18

KeyboardInterrupt: 

In [31]:
testloader = DataLoader(testset, batch_size=1)
acc_count = 0
count = 0
for i, (feature, label) in enumerate(testloader):
    feature = feature.to(device)
    label = label.to(device)
    prediction = perceiver(feature)
    prediction = torch.argmax(prediction)
    
    if prediction == label:
        acc_count += 1
        
    count += 1
    
    if i == 500:
        break
        
print(acc_count / count)

0.20359281437125748
